# Compiling `crc` using a notebook

This is just an insightful gist that shows how I automate some tasks.


## Setup host

this step prepares the host machine with `podman` and my dotfiles. The dotfiles contains several helper functions that simplify starting the containers.

In [ ]:
%%bash
if [ ! -d "$HOME/.dotfiles" ]; then
  curl -fsSL https://dotfiles.gbraad.nl/install.sh | sh
fi

To simplify executing cells with these dotfiles, we can load an extensin that sources the helper functions.

In [1]:
import sys, os
sys.path.append(os.path.expanduser("~/.dotfiles/ipython/.ipython/extensions"))
ip = get_ipython()
ip.extension_manager.load_extension("dotfiles")

dotfiles extension loaded!


## Prepare work directory

The source will be stored on the host machine in `~/Projects/crc-org/crc`. To contribute, it is expected your `ssh`-public key is registered with GitHub.

In [ ]:
%%dotscript
mkdir -p ~/Projects/crc-org
cd ~/Projects/crc-org
#git clone git@github.com:/crc-org/crc
git clone https://github.com/crc-org/crc

Cloning into 'crc'...
remote: Enumerating objects: 70554, done.
remote: Counting objects: 100% (1842/1842), done.
remote: Compressing objects: 100% (963/963), done.
remote: Total 70554 (delta 1399), reused 876 (delta 874), pack-reused 68712 (from 2)
Receiving objects: 100% (70554/70554), 63.58 MiB | 8.95 MiB/s, done.
Resolving deltas: 100% (37736/37736), done.
Updating files: 100% (12264/12264), done.


## Prepare developer environment

`devenv` is a helper function that starts developer environments using containers that are prepared, published to a public container registry.

The container here is a named `gofedora`, whose source and image is published to https://github.com/gbraad-devenv/fedora-golang, and `ghcr.io/gbraad-devenv/fedora-golang:latest`

The command `noinit` starts this image with `sleep infinity` to ensure the image remains active. Alternatively `system` can be used, which starts this image with `systemd`.

In [ ]:
%devenv gofedora rm

gofedorasys


In [ ]:
%devenv gofedora noinit

50ffdbc78ac41e74897f53ae5fdd275d4cded08a1bef76b85435943d1fa7ea66


## Compilation

The actual compilation process is performed from the container that was set up in the previous step. It performs a `make clean` and `make`. The host machine will not store any intermediate files, only the result.

In [ ]:
%devenv gofedora exec su gbraad -l -c "cd ~/Projects/crc-org/crc && make clean && make cross"

rm -f packaging/darwin/Distribution
rm -f packaging/darwin/Resources/welcome.html
rm -f packaging/darwin/scripts/postinstall
rm -rf packaging/darwin/root-crc/
rm -rf packaging/windows/msi
rm -f out/linux-amd64/split
rm -f packaging/rpm/crc.spec images/rpmbuild/Containerfile
rm -rf out
rm -f /home/gbraad/go/bin/crc
rm -rf release
GOARCH=arm64 GOOS=darwin go build -tags "containers_image_openpgp" -ldflags="-X github.com/crc-org/crc/v2/pkg/crc/version.crcVersion=2.50.0 -X github.com/crc-org/crc/v2/pkg/crc/version.ocpVersion=4.18.2 -X github.com/crc-org/crc/v2/pkg/crc/version.okdVersion=4.15.0-0.okd-2024-02-23-163410 -X github.com/crc-org/crc/v2/pkg/crc/version.microshiftVersion=4.18.2 -X github.com/crc-org/crc/v2/pkg/crc/version.commitSha=a85523 " -o out/macos-arm64/crc  ./cmd/crc
GOARCH=amd64 GOOS=darwin go build -tags "containers_image_openpgp" -ldflags="-X github.com/crc-org/crc/v2/pkg/crc/version.crcVersion=2.50.0 -X github.com/crc-org/crc/v2/pkg/crc/version.ocpVersion=4.18.2 -X githu

rm -f packaging/darwin/Distribution
rm -f packaging/darwin/Resources/welcome.html
rm -f packaging/darwin/scripts/postinstall
rm -rf packaging/darwin/root-crc/
rm -rf packaging/windows/msi
rm -f out/linux-amd64/split
rm -f packaging/rpm/crc.spec images/rpmbuild/Containerfile
rm -rf out
rm -f /home/gbraad/go/bin/crc
rm -rf release
go install -tags "containers_image_openpgp"  -ldflags="-X github.com/crc-org/crc/v2/pkg/crc/version.crcVersion=2.50.0 -X github.com/crc-org/crc/v2/pkg/crc/version.ocpVersion=4.18.2 -X github.com/crc-org/crc/v2/pkg/crc/version.okdVersion=4.15.0-0.okd-2024-02-23-163410 -X github.com/crc-org/crc/v2/pkg/crc/version.microshiftVersion=4.18.2 -X github.com/crc-org/crc/v2/pkg/crc/version.commitSha=a85523 "  ./cmd/crc


In [ ]:
%%dotscript
cd ~/Projects/crc-org/crc
git pull

Already up to date.


In [ ]:
%%dotscript
cd ~/Projects/crc-org/crc
out/linux-amd64/crc version
cp -r out/* ~/Uploads/

CRC version: 2.50.0+a85523
OpenShift version: 4.18.2
MicroShift version: 4.18.2
